In [169]:
import numpy as np
from tifffile import imread, imwrite 
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import glob
sys.path.append("../NEAT")
from  NEATUtils import Augmentation
from NEATUtils import MovieCreator, npzfileGenerator
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile
import scipy.ndimage as ndi
import csv
from stardist.models import StarDist2D
from csbdeep.models import Config, CARE
from csbdeep.utils import normalize
from skimage.filters import threshold_otsu
import cv2
from skimage.measure import regionprops
from skimage import measure, morphology
from skimage.filters import sobel
from skimage.segmentation import watershed
import matplotlib.pyplot  as plt

In [170]:
#Normal : Label 0 Apoptosis : Label 1 Division : Label 2 Macrocheate : Label 3 Non MatureP1 : Label 4 Mature Event: Label 5

In [171]:
ModelDirectory = '/home/sancere/VarunNewton/CurieDeepLearningModels/O-NEATweights/'

DenoisingModel = 'BorialisS1S2FlorisMidNoiseModel'
SegmentationMaskModel = 'DrosophilaMaskSegmentationCARE'
SegmentationStarModel = 'DrosophilaSegmentationSmartSeeds'

model = StarDist2D(config = None, name = SegmentationStarModel, basedir = ModelDirectory)
UnetModel = CARE(config = None, name = SegmentationMaskModel, basedir = ModelDirectory)
NoiseModel = CARE(config = None, name = DenoisingModel, basedir = ModelDirectory)

Loading network weights from 'weights_best.h5'.
Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


In [201]:
NoShiftNormal = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicNormalEventTrainingDataBin2/ResizeOriginal/'
ShiftRXYApoptosis = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicMasterApoptosisEventTrainingDataBin2/ShiftRXY/'
ShiftRXDivision = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicMasterDivisionEventTrainingDataBin2/ShiftRX/'
ShiftLXMacro = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicMacroKittyEventTrainingDataBin2/ShiftLX/'
ShiftLXYNonMature = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicNonMatureP1EventTrainingDataBin2/ShiftLXY/'
ShiftDYMature = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicMatureP1EventTrainingDataBin2/ShiftDY/'
#Look at center cell Label

In [202]:
Dir = ShiftDYMature

axis_norm = (0,1)
TotalCategories = 6

cellX = 0.5
cellY = 0.6851851851851852
trainlabel = 5


Label = np.zeros([TotalCategories + 5])
Label[trainlabel] = 1

Subdir = next(os.walk(Dir))
for x in Subdir[1]:
    currentdir = Dir  + x
    Path(currentdir + '/Segmentation/').mkdir(exist_ok = True)
    Raw_path = os.path.join(currentdir, '*tif')
    X = glob.glob(Raw_path)
    for fname in X:
        image = imread(fname)
        #Get Name
        Name = os.path.basename(os.path.splitext(fname)[0])



        x = image[image.shape[0]//2,:]

        originalX = x.shape[0]
        originalY = x.shape[1]
       
        #Stardist Prediction

        scale_percent = 400 # percent of original size
        width = int(x.shape[1] * scale_percent / 100)
        height = int(x.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        x = cv2.resize(x, dim)
        x = normalize(x,1,99.8,axis=axis_norm)
       

        MidImage, details = model.predict_instances(x)
       
        

        segimage = MidImage
       

        #Save different method segmentation

        imwrite((currentdir + '/Segmentation/'  + Name+ '.tif' ) , segimage.astype('float32'))
     
        smallimg = segimage.astype('uint16')
        centralLabel = smallimg[int(smallimg.shape[0]*cellY), int(smallimg.shape[1]*cellX)]
        
        
        if len(regionprops(smallimg)) > 0:
         for region in regionprops(smallimg):
           
            if region.label == centralLabel and centralLabel > 0:
              minr, minc, maxr, maxc = region.bbox
              
              center = region.centroid
              height = 0.25 * abs(maxc - minc)
              width = 0.25 * abs(maxr - minr)
            if centralLabel == 0:
            
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
               
        else:
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
            
              
        print(centralLabel, Name, 0.25 * center[0]/originalY, 0.25 * center[1]/originalX, height/originalY, width/originalX)
            
        Label[TotalCategories] = 0.25 * center[1]/originalX
        Label[TotalCategories + 1] = 0.25 * center[0]/originalY
        Label[TotalCategories + 2] = 0.5
        Label[TotalCategories + 3] = height/originalY
        Label[TotalCategories + 4] = width/originalX
        
        writer = csv.writer(open(currentdir + '/' + (Name) + 'DetectionLabel'  +".csv", "w"))
   
        for l in Label : writer.writerow ([l])
        
        Label[TotalCategories + 2] = 1
        Predwriter = csv.writer(open(currentdir + '/' + (Name) + 'PredictionLabel'  +".csv", "w"))
   
        for l in Label : Predwriter.writerow ([l])

6 127Bin2 0.6448633156966491 0.5001543209876543 0.06481481481481481 0.09722222222222222
5 102Bin2 0.6773648916124334 0.4310563235317404 0.25 0.2824074074074074
13 109Bin2 0.7504465637299425 0.4911797355939045 0.27314814814814814 0.2037037037037037
13 309Bin2 0.6647074886958289 0.4269325052075395 0.17592592592592593 0.25925925925925924
3 349Bin2 0.70369122491764 0.3936121094139962 0.20833333333333334 0.23148148148148148
5 39Bin2 0.6603703703703703 0.5258641975308642 0.07407407407407407 0.09259259259259259
0 168Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
13 51Bin2 0.6438746438746438 0.4750830959164293 0.09722222222222222 0.12037037037037036
8 253Bin2 0.6455387205387205 0.49450757575757576 0.0787037037037037 0.09259259259259259
1 114Bin2 0.6771978021978022 0.48499185999185995 0.07407407407407407 0.10648148148148148
0 106Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
4 30Bin2 0.6488547758284601 0.4736598440545809 0.125 0.09722222222222222
0 294B

12 282Bin2 0.7115953726123218 0.5372724419334589 0.12037037037037036 0.12037037037037036
15 177Bin2 0.7364995238811206 0.5530371372725905 0.1574074074074074 0.1388888888888889
8 231Bin2 0.7278672724769379 0.46095089682450274 0.30092592592592593 0.3055555555555556
6 230Bin2 0.6494575862112539 0.4303012497314783 0.2777777777777778 0.28703703703703703
0 31Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
12 204Bin2 0.7202552738267024 0.45836769051054765 0.1712962962962963 0.2222222222222222
0 227Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
2 58Bin2 0.6532921810699589 0.5125477660199882 0.09722222222222222 0.09259259259259259
13 296Bin2 0.7746966249525976 0.46786647284371974 0.2037037037037037 0.18055555555555555
13 302Bin2 0.6602925389157273 0.503858024691358 0.08333333333333333 0.09259259259259259
0 11Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
10 91Bin2 0.6505537899468959 0.46635327112958214 0.25925925925925924 0.287037037

17 279Bin2 0.643367223432583 0.4796255950940047 0.12962962962962962 0.09722222222222222
0 164Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
27 245Bin2 0.6953919922669923 0.461379477004477 0.125 0.1712962962962963
0 266Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
16 220Bin2 0.676714664511372 0.39603539220690825 0.26851851851851855 0.27314814814814814
17 157Bin2 0.6923243336794739 0.5308281412253375 0.11574074074074074 0.1111111111111111
15 345Bin2 0.780020742740041 0.49467061528465034 0.2916666666666667 0.21296296296296297
7 37Bin2 0.7870199821388703 0.4944943687827144 0.3194444444444444 0.2777777777777778
1 356Bin2 0.6316058595909342 0.48996683250414597 0.10648148148148148 0.11574074074074074
9 99Bin2 0.7338885534084809 0.545751140633387 0.25925925925925924 0.18518518518518517
13 308Bin2 0.6647074886958289 0.4269325052075395 0.17592592592592593 0.25925925925925924
0 62Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
0 346Bi

3 42Bin2 0.6656440172490227 0.4170444928061903 0.18981481481481483 0.20833333333333334
0 64Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
7 192Bin2 0.7919652434132843 0.6074380537753621 0.37962962962962965 0.2916666666666667
10 143Bin2 0.6731832543443917 0.4093901096270765 0.25925925925925924 0.25925925925925924
4 12Bin2 0.7558475141808475 0.4828244911578245 0.25925925925925924 0.1712962962962963
11 82Bin2 0.7307176584394065 0.36018566407110497 0.2962962962962963 0.35185185185185186
0 353Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
0 237Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
13 219Bin2 0.6199694360190124 0.37900208021011283 0.2916666666666667 0.30092592592592593
18 139Bin2 0.6683532554559449 0.4823077967943494 0.09259259259259259 0.12037037037037036
13 337Bin2 0.6251555172743802 0.5012680639295626 0.07407407407407407 0.1527777777777778
10 96Bin2 0.7709944039466903 0.5151038215153524 0.24074074074074073 0.180555555

12 181Bin2 0.6528932355338223 0.4595309245675994 0.18518518518518517 0.2916666666666667
0 35Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
0 244Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
13 240Bin2 0.6511163200057477 0.4433667421058304 0.23148148148148148 0.2638888888888889
16 327Bin2 0.6688539853648494 0.4081723841071997 0.2175925925925926 0.24074074074074073
5 90Bin2 0.6883090614886732 0.5140911542610572 0.06481481481481481 0.10185185185185185
1 127Bin2 0.6411146723646723 0.5001780626780626 0.06018518518518518 0.10185185185185185
5 102Bin2 0.6777837132003798 0.43183760683760686 0.25 0.2824074074074074
12 109Bin2 0.7508282480414074 0.4909144462570238 0.27314814814814814 0.2037037037037037
11 309Bin2 0.6661843474729012 0.42771558398700277 0.17592592592592593 0.2638888888888889
3 349Bin2 0.7033308476261496 0.3932109122545364 0.20833333333333334 0.23148148148148148
3 39Bin2 0.6600050276520866 0.5264370705547177 0.06944444444444445 0.092592592

0 239Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
9 88Bin2 0.7789183807219198 0.4863673791759091 0.2824074074074074 0.19907407407407407
2 328Bin2 0.6684960036023866 0.509526623888326 0.0787037037037037 0.11574074074074074
8 134Bin2 0.5888646215126107 0.4667725226356044 0.22685185185185186 0.25925925925925924
14 76Bin2 0.6066491472995538 0.49427883167720565 0.1712962962962963 0.22685185185185186
19 380Bin2 0.7273259414064012 0.5692557761523278 0.16666666666666666 0.17592592592592593
13 282Bin2 0.711338141025641 0.53731525997151 0.12037037037037036 0.12037037037037036
14 177Bin2 0.7404888344226579 0.5418572984749456 0.1527777777777778 0.1388888888888889
8 231Bin2 0.7283772476302159 0.45873491400371347 0.3055555555555556 0.3055555555555556
4 230Bin2 0.6504153394541253 0.4311036162638186 0.2777777777777778 0.28703703703703703
0 31Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
12 204Bin2 0.7210127965043696 0.45832032875572204 0.1712962962962963 0.

18 158Bin2 0.5932605491063202 0.48283754836926474 0.19907407407407407 0.22685185185185186
0 214Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
11 111Bin2 0.7897895727201799 0.41814103446693285 0.36574074074074076 0.3194444444444444
4 195Bin2 0.6758106654372215 0.5435876748117412 0.09722222222222222 0.14351851851851852
4 202Bin2 0.7683385411880036 0.475143981864412 0.3194444444444444 0.20833333333333334
0 105Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
15 279Bin2 0.6469474969474969 0.47979242979242975 0.12037037037037036 0.10185185185185185
0 164Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
26 245Bin2 0.6954324365405077 0.4612656432081877 0.125 0.1712962962962963
0 266Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
16 220Bin2 0.6769868089206268 0.39614327410807904 0.26851851851851855 0.27314814814814814
17 157Bin2 0.6925586914145266 0.5318459191456903 0.11574074074074074 0.1111111111111111
13 345Bin2 0.

0 290Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
3 377Bin2 0.6733870967741935 0.5239446435682995 0.0787037037037037 0.06944444444444445
5 393Bin2 0.6611159801343851 0.36047678557903506 0.2962962962962963 0.33796296296296297
6 49Bin2 0.7580852510571385 0.4452572193536049 0.2638888888888889 0.18518518518518517
0 13Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
3 42Bin2 0.6646510456034266 0.41688187620727307 0.19444444444444445 0.2037037037037037
0 64Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
9 192Bin2 0.7911935658286023 0.6097910071597381 0.38425925925925924 0.2962962962962963
9 143Bin2 0.6731962947046746 0.4096257301564564 0.25462962962962965 0.25925925925925924
4 12Bin2 0.755736531986532 0.4830723905723906 0.2638888888888889 0.1712962962962963
11 82Bin2 0.7317217370002921 0.3607636827873409 0.2962962962962963 0.35185185185185186
0 353Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
0 237Bin2 0.6851

10 373Bin2 0.6910934744268077 0.4735008818342152 0.06944444444444445 0.08333333333333333
7 165Bin2 0.7312820896266388 0.4983404782081319 0.24537037037037038 0.26851851851851855
0 265Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
3 335Bin2 0.7233264814203038 0.41326212238006094 0.35185185185185186 0.25462962962962965
4 65Bin2 0.6833133573045679 0.3764112694396354 0.26851851851851855 0.37962962962962965
11 181Bin2 0.6539094152235487 0.46003415080133375 0.18981481481481483 0.2916666666666667
0 35Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
0 244Bin2 0.6851851851851852 0.5 0.18518518518518517 0.18518518518518517
14 240Bin2 0.6510455881825679 0.44357722921372483 0.23148148148148148 0.2638888888888889
15 327Bin2 0.6691302246021121 0.40817668823441916 0.2175925925925926 0.24074074074074073
5 90Bin2 0.6884057971014492 0.5139783503310074 0.06481481481481481 0.10185185185185185
